In [56]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns
import re
from sklearn.model_selection import train_test_split

pd.set_option('display.max_columns', 100)
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')

In [57]:
test.head()

,ID,DATOP,FLTID,DEPSTN,ARRSTN,STD,STA,STATUS,AC
0,test_id_0,2016-05-04,TU 0700,DJE,TUN,2016-05-04 06:40:00,2016-05-04 07.30.00,ATA,TU 32AIMF
1,test_id_1,2016-05-05,TU 0395,TUN,BKO,2016-05-05 15:20:00,2016-05-05 20.05.00,ATA,TU 320IMW
2,test_id_2,2016-05-06,TU 0745,FRA,TUN,2016-05-06 10:00:00,2016-05-06 12.25.00,ATA,TU 32AIMC
3,test_id_3,2016-05-11,TU 0848,BEY,TUN,2016-05-11 09:40:00,2016-05-11 13.10.00,ATA,TU 31BIMO
4,test_id_4,2016-05-11,TU 0635,ORY,MIR,2016-05-11 09:50:00,2016-05-11 12.35.00,ATA,TU 736IOQ


In [58]:
#deleting ID 
train=train.drop('ID',axis = 1)
#checking if there is any missing data 
print('train missing data')
print(train.isnull().sum())
print('///////////')
print('test missing data')
print(test.isnull().sum())
test_id_for_submission = test['ID']
test = test.drop('ID',axis= 1)
y=train['target']
train_with_no_target = train.drop('target',axis=1)

train missing data
DATOP     0
FLTID     0
DEPSTN    0
ARRSTN    0
STD       0
STA       0
STATUS    0
AC        0
target    0
dtype: int64
///////////
test missing data
ID        0
DATOP     0
FLTID     0
DEPSTN    0
ARRSTN    0
STD       0
STA       0
STATUS    0
AC        0
dtype: int64


In [59]:
#check if flight id is unique 
print('length of FLTID is ' ,len(train['FLTID'].unique()) )
# Flight id is composed of ; name of the airline company plus an id 


length of FLTID is  1861


In [60]:
#concatening train and test 
all_data = pd.concat([train_with_no_target,test],ignore_index=True)
all_data.head()

,DATOP,FLTID,DEPSTN,ARRSTN,STD,STA,STATUS,AC
0,2016-01-03,TU 0712,CMN,TUN,2016-01-03 10:30:00,2016-01-03 12.55.00,ATA,TU 32AIMN
1,2016-01-13,TU 0757,MXP,TUN,2016-01-13 15:05:00,2016-01-13 16.55.00,ATA,TU 31BIMO
2,2016-01-16,TU 0214,TUN,IST,2016-01-16 04:10:00,2016-01-16 06.45.00,ATA,TU 32AIMN
3,2016-01-17,TU 0480,DJE,NTE,2016-01-17 14:10:00,2016-01-17 17.00.00,ATA,TU 736IOK
4,2016-01-17,TU 0338,TUN,ALG,2016-01-17 14:30:00,2016-01-17 15.50.00,ATA,TU 320IMU


In [61]:
all_data['STA'] = all_data['STA'].apply(lambda x : re.sub('\.',':',x) )

In [62]:
all_data['STA'] = pd.to_datetime(all_data['STA'])
all_data['STD'] = pd.to_datetime(all_data['STD'])

# Adding duration feature in minutes 

In [63]:
duration=[]
for e1,e2 in zip(all_data.STA,all_data.STD):
    duration.append(((e1-e2).seconds)/60)
    
all_data['Duration']=duration

#  dealing with DATOP STD STA 

In [64]:
#defining function to split DATOP into interesting features 
# function from https://gist.github.com/adidoit/d32f6117647b0e8564edc6507f9e3625 with some changes 
import re
def add_datepart(df, fldname, drop=True, time=False, errors="raise"):
    """add_datepart converts a column of df from a datetime64 to many columns containing
    the information from the date. This applies changes inplace.
      
    """
    fld = df[fldname]
    fld_dtype = fld.dtype
    if isinstance(fld_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
        fld_dtype = np.datetime64

    if not np.issubdtype(fld_dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True, errors=errors)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    if time: attr = attr + ['Hour', 'Minute', 'Second']
    for n in attr: df[targ_pre + n] = getattr(fld.dt, n.lower())
    col = [fldname+'Is_month_end',fldname+'Is_month_start',fldname+'Is_quarter_end',fldname+'Is_quarter_start',fldname+'Is_year_end',fldname+'Is_year_start']
    di = {True : 1,False:0}
    df[col]=df[col].replace(di)
        
    if drop: df.drop(fldname, axis=1, inplace=True)

In [65]:
#defining function to split STA and STD To minutes / hours / seconds 
def split_date(df,fn):
    fld = df[fn]
    fld
    col_to_add = [fn+'_Hour', fn +'_Minutes']
    H=[]
    M=[]
    S=[]
    
    for entry in fld:
        
        
        hours = entry.hour
        minutes = entry.minute
        H.append(hours)
        M.append(minutes)

    df[fn+'_Hour'] = H
    df[fn +'_Minutes'] = M 
    df[col_to_add] = df[col_to_add].astype('int')
    df.drop(fn,axis = 1,inplace = True)

In [66]:
split_date(all_data,'STA')
split_date(all_data,'STD')
add_datepart(all_data, 'DATOP')

In [67]:
#Extracting from FLTID , name of company 
all_data['company name'] = all_data['FLTID'].apply(lambda x: x.split(' ')[0] )

In [68]:
#adding datais to have a score 
col = ['DATOPIs_month_end','DATOPIs_month_start','DATOPIs_quarter_end','DATOPIs_quarter_start','DATOPIs_year_end','DATOPIs_year_start']  
z=[]
for index, row in all_data.iterrows():
    su = 0
    for h in col:
        su=su+row[h]
    z.append(su)
        
all_data['SUM'] = z

In [69]:
col_label_encoder=['FLTID','company name','AC','DATOPYear']
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
for cola in col_label_encoder:
    all_data[cola] = labelencoder.fit_transform(all_data[cola])

In [70]:
from sklearn.preprocessing import OneHotEncoder
cat_oh = ['STATUS','ARRSTN','DEPSTN']
# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols= pd.DataFrame(OH_encoder.fit_transform(all_data[cat_oh]))

# One-hot encoding removed index:
OH_cols.index = all_data.index
# Remove categorical columns (will replace with one-hot encoding)
num_data = all_data.drop(cat_oh, axis=1)

# Add one-hot encoded columns to numerical features
OH_data = pd.concat([num_data, OH_cols], axis=1)

all_data=OH_data

In [71]:
all_data.head()

,FLTID,AC,Duration,STA_Hour,STA_Minutes,STD_Hour,STD_Minutes,DATOPYear,DATOPMonth,DATOPWeek,DATOPDay,DATOPDayofweek,DATOPDayofyear,DATOPIs_month_end,DATOPIs_month_start,DATOPIs_quarter_end,DATOPIs_quarter_start,DATOPIs_year_end,DATOPIs_year_start,company name,SUM,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,...,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268
0,239,47,145.0,12,55,10,30,0,1,53,3,6,3,0,0,0,0,0,0,14,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,266,29,110.0,16,55,15,5,0,1,2,13,2,13,0,0,0,0,0,0,14,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,93,47,155.0,6,45,4,10,0,1,2,16,5,16,0,0,0,0,0,0,14,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,173,51,170.0,17,0,14,10,0,1,2,17,6,17,0,0,0,0,0,0,14,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,145,35,80.0,15,50,14,30,0,1,2,17,6,17,0,0,0,0,0,0,14,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [72]:
train_data = all_data[:len(train)]
test_data = all_data[len(train):]
train_data['target']=y
#ive done it to save files 
train_data.to_csv('train_data.csv',index=False)
test_data.to_csv('test_data.csv',index=False)

C:\Users\ASUS\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [73]:
train = pd.read_csv('train_data.csv')
test = pd.read_csv('test_data.csv')
test_for_id = pd.read_csv('Test.csv')
idd = test_for_id['ID']
y=train['target']
X=train.drop('target',axis=1)

# Trying corrrelations

In [47]:
#X.drop('DATOPDayofyear',axis=1,inplace=True)
#test.drop('DATOPDayofyear',axis=1,inplace=True)

# Modeling

In [48]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2, random_state=32)

In [49]:
def evaluate(model, test_features, test_labels):
    
    from sklearn.metrics import mean_squared_error
    predictions = model.predict(test_features)
    rmse = np.sqrt(mean_squared_error(test_labels , predictions ))
    print('Model Performance')
    print('rmse = {:0.4f}.'.format(rmse))

# XGBRegressor with gridsearchCV

In [75]:
from sklearn.metrics import mean_squared_error
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import GridSearchCV

In [51]:
xgb_model = XGBRegressor()
parameters = {
              'objective':['reg:linear'],
              'learning_rate': [0.025], #so called `eta` value
              'max_depth': [7,8],
              'min_child_weight': [4],
              'subsample': [0.9],
              'colsample_bytree': [0.7],
              'n_estimators': [200,300],
               'silent':[1],
                'nthread': [4]}

xgb_grid = GridSearchCV(xgb_model,parameters,cv = 5,verbose=True,scoring='neg_mean_squared_error')
xgb_grid.fit(X,y)
print(xgb_grid.best_params_)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\ASUS\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\ASUS\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\ASUS\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\ASUS\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\ASUS\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future ver

{'colsample_bytree': 0.7, 'learning_rate': 0.025, 'max_depth': 8, 'min_child_weight': 4, 'n_estimators': 300, 'nthread': 4, 'objective': 'reg:linear', 'silent': 1, 'subsample': 0.9}


In [77]:
param= {'colsample_bytree': 0.7, 'learning_rate': 0.025, 'max_depth': 8,
        'min_child_weight': 4, 'n_estimators': 200, 'nthread': 4,
        'objective': 'reg:linear', 'silent': 1, 'subsample': 0.9}
xgb_other = XGBRegressor(**param)
xgb_other.fit(X,y)
pred = xgb_other.predict(test)


# LASSO

In [ ]:
"""from sklearn import linear_model
lasso_params = {'alpha':[0.02, 0.024, 0.025, 0.026, 0.03],
                'max_iter':[500,100,1500],
                'normalize':[True,False],
                
               }
lasso = linear_model.Lasso()
lasso_grid = GridSearchCV(lasso ,lasso_params,cv = 5,verbose=True,scoring='neg_mean_squared_error')
lasso_grid.fit(X,y)
print(lasso_grid.best_params_)
lasso_best =linear_model.Lasso(**lasso_grid.best_params_) 
lasso_best.fit(X_train,y_train)
lasso_best.fit(X,y)
pred = lasso_best.predict(test)"""

# PREP SUBMISSION

In [78]:
submit=pd.DataFrame()
submit['ID']=idd
submit['target']=pred

submit.head()
submit['target']=submit['target'].astype('int')
submit.head()
submit.to_csv('subb.csv',index=False)